# Purpose of Code
The following code will scrape and download the most recent FEC filing made by each candidate. Selenium will be used to automate the browser, and is needed so that the javascript of the web page is executed. Pandas will be used for data manipulation. OS.path and time will allow the code to wait for downloads to finish.

In [50]:
from selenium import webdriver
import pandas as pd
import time
import os.path

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import chromedriver_autoinstaller

The current version of the scraper is fetching Tim Ryan and J.D. Vance campaign finance data.

In [51]:
ryan_url = 'https://www.fec.gov/data/reports/house-senate/?committee_id=C00777771&is_amended=false&data_type=processed&candidate_id=S2OH00402&cycle=2022'
vance_url = 'https://www.fec.gov/data/reports/house-senate/?committee_id=C00772947&committee_id=C00783142&is_amended=false&data_type=processed&candidate_id=S2OH00436&cycle=2022'

Preferences are needed to ensure downloads go the right directory

In [52]:
chromedriver_autoinstaller.install()
prefs = {"download.default_directory" : "/users/jason/Documents/Github/Ohio-Senate-Funding"};
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs",prefs)

In [53]:
driver = webdriver.Chrome(options=options)
driver.get(ryan_url)
driver.implicitly_wait(5)

The code parses the website HTML, and uses the xpath function to find all elements with the text '.csv.' By viewing the structure of the web page, I know that the first element of this list will be the most recent filing made by the candidate.

In [54]:
links = driver.find_elements_by_xpath("//*[contains(text(), '.csv')]")
ryan_link = links[0].get_attribute('href')
ryan_link

'https://docquery.fec.gov/csv/647/1627647.csv'

If one opens the link above, they will see that the page is essentially a text file that can be downloaded, but because the link does not *initiate* a download, traditional *request* and *read_csv* methods cannot be used. So, the driver.get() method is used to initiate the download into the directory specified earlier.

In [55]:
ryan_file = ryan_link[ryan_link.rfind('/')+1:]
driver.get(ryan_link)

while not os.path.exists('/users/jason/Documents/Github/Ohio-Senate-Funding/{}'.format(ryan_file)):
    time.sleep(1)

Now, the code repeats all of the above steps for the next candidates data.

In [56]:
driver.get(vance_url)
driver.implicitly_wait(5)

In [57]:
links = driver.find_elements_by_xpath("//*[contains(text(), '.csv')]")
vance_link = links[0].get_attribute('href')
vance_link

'https://docquery.fec.gov/csv/061/1628061.csv'

In [58]:
vance_file = vance_link[vance_link.rfind('/')+1:]
driver.get(vance_link)

while not os.path.exists('/users/jason/Documents/Github/Ohio-Senate-Funding/{}'.format(vance_file)):
    time.sleep(1)

# Selenium is no longer needed
driver.close()

The data from FEC is pretty messy, the following cells are an attempt to clean it up. From here on out, it is crucial to understand the dimensions of the CSV file and grab the necessary data. 
The cell directly below was inspired by the link in the comment at the top of the cell. The FEC data is stored in columns with conflicting data types, so the functions convert the columns with majority integers or strings to the majority data type.

In [59]:
# Helpful guide https://www.roelpeters.be/solved-dtypewarning-columns-have-mixed-types-specify-dtype-option-on-import-or-set-low-memory-in-pandas/

def convert_string(x):
    if not x:
        return ''
    try:
        return str(x)   
    except:        
        return ''

def convert_int(x):
    if not x:
        return ''
    try:
        return str(x)   
    except:        
        return ''

convert_dict = {10:convert_string, 18:convert_string, 22:convert_int, 27:convert_string, 28:convert_string, 29:convert_string, 30:convert_string, 32:convert_string, 33:convert_string, 34:convert_string, 41:convert_string}

ryan = pd.read_csv(ryan_file, converters=convert_dict, header=1)
vance = pd.read_csv(vance_file, converters=convert_dict, header=1)

os.remove(ryan_file)
os.remove(vance_file)

Now, the data comprehension is again very important. Here, the *relevant* data is gathered into a dataframe for each candidate.

In [60]:
ryan_details = pd.DataFrame()
ryan_details['Type'] = ryan.iloc[:,5]
ryan_details['Address'] = ryan.iloc[:,12]
ryan_details['City'] = ryan.iloc[:,14]
ryan_details['State'] = ryan.iloc[:,15]
ryan_details['Amount'] = ryan.fillna(0).iloc[:,20]
ryan_details = ryan_details.iloc[1:]
ryan_details.head()

,Type,Address,City,State,Amount
1,PAC,PO Box 382110,Cambridge,MA,50.0
2,IND,3837 Garrison St NW,Washington,DC,400.0
3,PAC,PO Box 382110,Cambridge,MA,400.0
4,IND,5417 S Oneida Way,Greenwood Village,CO,500.0
5,PAC,PO Box 382110,Cambridge,MA,500.0


In [64]:
vance_details = pd.DataFrame()
vance_details['Type'] = vance.iloc[:,5]
vance_details['Address'] = vance.iloc[:,12]
vance_details['City'] = vance.iloc[:,14]
vance_details['State'] = vance.iloc[:,15]
vance_details['Amount'] = vance.fillna(0).iloc[:,20]
vance_details = vance_details.iloc[3:]
vance_details.head()

,Type,Address,City,State,Amount
3,IND,70539 CHERMONT ROAD,BRIDGEPORT,OH,100.0
4,PAC,PO BOX 9891,ARLINGTON,VA,115.5
5,IND,2547 WALTERS AVENUE,NORTHBROOK,IL,25.0
6,PAC,PO BOX 9891,ARLINGTON,VA,115.0
7,IND,617 HUNTS GROVE DRIVE,NORTH AUGUSTA,SC,10.0


The scraper has now completed its job. The sums for the contributions are given

In [69]:
print("Total for Ryan:\t {}".format(round(sum(ryan_details['Amount']),0)))
print("Total for J.D.:\t {}".format(round(sum(vance_details['Amount']),0)))

Total for Ryan:	 20090921.0
Total for J.D.:	 3569365.0


In [65]:
ryan_details.to_csv('data/tim_ryan.csv')
vance_details.to_csv('data/jd_vance.csv')